# Inverse Kinematics
In this section we will implement a method for the inverse kinematics. It will estimate the joint angles based on the position and orientation of the end effector. Today we will use the NLsolve library to solve the inverse kinematics problem. In later lectures we will learn how to implement our own inverse kinematics solver. 

In [ ]:
Pkg.add("NLsolve")
Pkg.add("LinearAlgebra")

In [ ]:
using NLsolve
using LinearAlgebra

To estimate the joint angles we first define a function that we want to solve. This function takes the joint angles as input and returns the difference between the current position and the desired position of the end effector. We want to find a point where this difference is zero. 

Therefore we set an initial guess and use the NLsolve library to solve the function.

In [ ]:
function inverse_kinematics(desired_pos)
    # Define the function to be solved
    function f!(F, q)
        # Calculate the forward kinematics for the current joint angles
        current_pos_heading = complete_forward_kinematics(q)
        # The current position of the end effector
        current_pos = current_pos_heading[1]
        # Calculate the error between the current position and the desired position
        F[1] = current_pos[1] - desired_pos[1]
        F[2] = current_pos[2] - desired_pos[2]
        F[3] = current_pos[3] - desired_pos[3]
    end

    # Set an initial guess for the joint angles
    q0 = [0.0, 0.0, 0.0, 0.0]

    # Solve for the joint angles using NLsolve
    result = nlsolve(f!, q0)

    # Check if the solver converged
    if !result.f_converged
        println("Solver did not converge!")
        println(result)
        return nothing
    end

    # Return the joint angles
    println(result)
    return result.zero
end

Let's test our inverse kinematics function by giving it a desired position and seeing if it can find the joint angles that will move the end effector to that position. Let's use the position we estimated erlier.

In [ ]:
pos = pos_heading[1]

In [ ]:
inverse_kinematics(pos) # 0.6, 0.6, -0.5, -0.1

Well this is not the position we wanted. The reason for this is that the inverse kinematics solver finds a solution to the problem we provided. We did however not provide the heading of the end effector and therefore the solver found a solution where the end effector is pointing in a different direction than the one we wanted. 

Let us define a new ```inverse_kinematics``` function that takes the desired position and orientation as input and returns the joint angles that will move the end effector to that position and orientation. Since we already have a function that calculates the forward kinematics and gives us a position and orientation we can use that to calculate the current position and orientation of the end effector and set a fourth constraint for the inverse kinematics solver. Keep in mind that we only have 4 joints and therefore we can only set 4 constraints.

In [ ]:
function inverse_kinematics(desired_pos, desired_orient)
    # Define the function to be solved
    function inv_f!(F, q)
        # Calculate the forward kinematics for the current joint angles
        current_pos_heading = complete_forward_kinematics(q)
        # The current position of the end effector
        current_pos = current_pos_heading[1]
        # Calculate the error between the current position and the desired position
        F[1] = current_pos[1] - desired_pos[1]
        F[2] = current_pos[2] - desired_pos[2]
        F[3] = current_pos[3] - desired_pos[3]

        # The current orientation of the end effector
        current_orient = current_pos_heading[2]
        # Calculate the error between the current orientation and the desired orientation
        F[4] = (current_orient[1] - desired_orient[1])^2 + (current_orient[2] - desired_orient[2])^2 + (current_orient[3] - desired_orient[3])^2
    end

    # Set an initial guess for the joint angles
    q0 = [0.0, 0.0, 0.0, 0.0]

    # Solve for the joint angles using NLsolve
    result = nlsolve(inv_f!, q0)

    # Check if the solver converged
    if !result.f_converged
        println("Solver did not converge")
        return nothing
    end

    # Return the joint angles
    println(result)
    return result.zero
end

Let's test our new inverse kinematics function by giving it a desired position and orientation and seeing if it can find the joint angles that will move the end effector to that position and orientation.

In [ ]:
pos = pos_heading[1]
heading = pos_heading[2]
inverse_kinematics(pos, heading) # 0.6, 0.6, -0.5, -0.1

That looks better. We can now use this method to move the end effector to any position and orientation we want.

However, there is one problem. The inverse kinematics solver can only find a solution if the desired position and orientation is reachable. If we give it a position and orientation that is not reachable it will not find a solution. Let's test this by giving it a position and orientation that is not reachable.

In [ ]:
inverse_kinematics([0.3, 0.3, 0.16], [0.0, 0.0, 0.0])

There is another Problem. The inverse kinematics solver can find a solution even if the robot itself is not able to reach the desired position. This may be for example due to constraints of the maximum rotation of the joints.

Let's test this by giving it a position that is reachable but the robot itself is not able to reach. We can find one by checking the limits of the joints and using the forward kinematic solver to find the position of the end effector for the maximum rotation of the joints.
We have the following limits: 

Joint 1:
```<limit velocity="4.8" effort="1" lower="${-pi*0.9}" upper="${pi*0.9}" />```

Joint 2:
```<limit velocity="4.8" effort="1" lower="${-pi*0.57}" upper="${pi*0.5}" />```

Joint 3:
```<limit velocity="4.8" effort="1" lower="${-pi*0.3}" upper="${pi*0.44}" />```

Joint 4:
```<limit velocity="4.8" effort="1" lower="${-pi*0.57}" upper="${pi*0.65}" />```

In [ ]:
pos_heading = complete_forward_kinematics([-π*1.2, 0.6, -0.5, -0.1]) # above limit for joint 1

In [ ]:
inverse_kinematics(pos_heading[1])

So we can see that we have to be careful with the results of our inverse kinematics solver. We have to check if the solution is reachable and if the robot itself is able to reach the desired position. 